# Predicting outcome of chess games based on my performance this year

In [18]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

from keras.models import Sequential
from keras.layers import Dense

from keras.wrappers.scikit_learn import KerasClassifier

from dummies_bins_test_train_cv import get_Xy_train_test

## If new data is added, uncomment and run the next line

In [19]:
# from clean_chess_game_log import main_cleanup
# _,_,_ = main_cleanup('../data/dest.pgn')

## Load the data to run predictions

In [20]:
df = pd.read_csv('../data/use_for_predictions.csv')

## Create Train and Test sets, along with clean the data
### Also, see the shape to know the data is in right shape

In [24]:
X_train, X_test, y_train, y_test, X, y, df_clean = get_Xy_train_test(df, .985, .995)

y Shape: (2104,)
.
X Shape: (2104, 29)
X_train Shape: (2078, 29)
X_test Shape: (26, 29)
y_train Shape: (2078,)
y_test Shape: (26,)


## Making sure the data is in the correct format

In [25]:
X_train = X_train.astype('float64')
X_test = X_test.astype('float64')
y_train = y_train.astype('int64')
y_test = y_test.astype('int64')

## Scale the data for easier computations

In [26]:
std_sclr = StandardScaler()
X_train = std_sclr.fit_transform(X_train)
X_test = std_sclr.fit_transform(X_test)

### See the scale of each value if interested

In [27]:
std_sclr.scale_

array([1.        , 0.49851852, 8.5408867 , 1.        , 1.        ,
       1.        , 0.31948553, 1.        , 0.42132504, 1.        ,
       0.19230769, 0.31948553, 0.26646936, 0.19230769, 0.31948553,
       0.26646936, 0.26646936, 0.26646936, 1.        , 0.19230769,
       1.        , 1.        , 1.        , 0.36080121, 0.31948553,
       0.26646936, 0.4435601 , 0.19230769, 0.36080121])

## Creating the classifier:
__The _Fun_ Part!!__

- __Step 1__
    - Add the sequential layer
- __Step 2__
    - Add the __Dense__ layer with following parameters:
        - Units = 128
        - Initializer = Uniform
        - Activation = Softmax (Since the outcome is either 1 or 0, Softmax works wonders)
        - Input Dim = the second part of the X Shape (# of columns)
- __Step 3__
    - Add the second __Dense__ layer with following parameters:
        - Units = 256 (Double it up!!)
        - Initializer = Normal (Throw a curve ball!)
        - Activation = ReLu (Prevents Vanashing Gradiant problem)
- __Step 4__
    - Add the third __Dense__ layer with following parameters:
        - Units = 64 (Make it easier for the compiling)
        - Initializer = Uniform
        - Activation = Softmax
- __Step 5__
    - Add the final __Dense__ layer with following parameters:
        - Units = 1 (Just need 1 column for output)
        - Initializer = Normal (#MuscleConfusion. I kid!)
        - Activation = Sigmoid
- __Step 6__
    - Only think left is to __COMPILE__ it. From all the parameters I've tested. These work the best:
        - Optimizer = Nadam (Nesterov Adam optimizer - Nadam is Adam RMSprop with Nesterov momentum)
        - loss = Mean Absolute Error (Didn't get great results with binary_crossentropy, even though it was my first choice. Mean Squared Error, made error very small, since the loss is between 0 and 1.)
        - metrics = Binary Accuracy (Only have 0 or 1 to predict)

In [28]:
def _classifier():
    classifier = Sequential()

    classifier.add(Dense(units=128, kernel_initializer='uniform',
                         activation='softmax', input_dim=X.shape[1]))

    classifier.add(
        Dense(units=256, kernel_initializer='normal', activation='relu'))

    classifier.add(
        Dense(units=64, kernel_initializer='uniform', activation='softmax'))

    classifier.add(
        Dense(units=1, kernel_initializer='normal', activation='sigmoid'))

    classifier.compile(
        optimizer='nadam', loss='mae', metrics=['accuracy'])

    return classifier

In [12]:
classifier = _classifier()

## Fitting the training data
#### After many iterations testing different batch sizes and epochs (Using grid search and for loops), the following parameters work the best:
- Batch Size = 6
- Epochs = 100
- Class Weight = Balanced
- Shuffle = False (Since it is a time series model)

In [14]:
classifier.fit(X_train, y_train, batch_size=4, epochs=50,
               class_weight='balanced', shuffle=False)

## Training done! Now let's get the predicted results

In [ ]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

## Create the Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

# 85.7% Accuracy!!!

In [ ]:
print(f'{((cm[0][0]+cm[1][1])/cm.sum()*100).round(1)}%')

In [29]:
results = {}
kfdict = {}

In [17]:
for bs in [20, 16, 12, 8, 6]:
    for ep in [10, 25, 50, 100, 200]:
#         classifier = KerasClassifier(_classifier, batch_size=bs, epochs=ep, shuffle=False)
#         kfmodel = cross_val_score(classifier, X_train, y_train, cv=1, n_jobs=-2, scoring='accuracy')
        classifier = _classifier()
        classifier.fit(X_train, y_train, batch_size=bs, epochs=ep, class_weight='balanced', shuffle=False)
        y_pred = classifier.predict(X_test)
        y_pred = (y_pred > 0.5)
        cm = confusion_matrix(y_test, y_pred)
        results[(f'bs:{bs}, ep:{ep} (cm)')] = cm
        results[(f'bs:{bs}, ep:{ep} (accuracy)')] = f'{((cm[0][0]+cm[1][1])/cm.sum()*100).round(1)}%'
#         kfdict[(f'bs:{bs}, ep:{ep} (mean)')] = kfmodel.mean()
#         kfdict[(f'bs:{bs}, ep:{ep} (std)')] = kfmodel.std()
        print(results)
        

Epoch 1/20
1152/1152 [==============================] - 2s 1ms/step - loss: 0.5002 - acc: 0.4896
Epoch 2/20
1152/1152 [==============================] - 1s 840us/step - loss: 0.4887 - acc: 0.5729
Epoch 3/20
1152/1152 [==============================] - 1s 847us/step - loss: 0.4164 - acc: 0.6970
Epoch 4/20
1152/1152 [==============================] - 1s 832us/step - loss: 0.3781 - acc: 0.7109
Epoch 5/20
1152/1152 [==============================] - 1s 854us/step - loss: 0.3481 - acc: 0.7283
Epoch 6/20
1152/1152 [==============================] - 1s 846us/step - loss: 0.3262 - acc: 0.7335
Epoch 7/20
1152/1152 [==============================] - 1s 829us/step - loss: 0.3133 - acc: 0.7361
Epoch 8/20
1152/1152 [==============================] - 1s 847us/step - loss: 0.3019 - acc: 0.7387
Epoch 9/20
1152/1152 [==============================] - 1s 826us/step - loss: 0.2967 - acc: 0.7318
Epoch 10/20
1152/1152 [==============================] - 1s 870us/step - loss: 0.2854 - acc: 0.7439
Epoch 11/20

Epoch 21/60
1152/1152 [==============================] - 1s 855us/step - loss: 0.2384 - acc: 0.7682
Epoch 22/60
1152/1152 [==============================] - 1s 889us/step - loss: 0.2404 - acc: 0.7665
Epoch 23/60
1152/1152 [==============================] - 1s 950us/step - loss: 0.2387 - acc: 0.7648
Epoch 24/60
1152/1152 [==============================] - 1s 849us/step - loss: 0.2307 - acc: 0.7743
Epoch 25/60
1152/1152 [==============================] - 1s 807us/step - loss: 0.2283 - acc: 0.7743
Epoch 26/60
1152/1152 [==============================] - 1s 805us/step - loss: 0.2265 - acc: 0.7752
Epoch 27/60
1152/1152 [==============================] - 1s 846us/step - loss: 0.2303 - acc: 0.7743
Epoch 28/60
1152/1152 [==============================] - 1s 834us/step - loss: 0.2241 - acc: 0.7786
Epoch 29/60
1152/1152 [==============================] - 1s 809us/step - loss: 0.2310 - acc: 0.7708
Epoch 30/60
1152/1152 [==============================] - 1s 934us/step - loss: 0.2299 - acc: 0.7726


1152/1152 [==============================] - 1s 832us/step - loss: 0.2256 - acc: 0.7743
Epoch 41/80
1152/1152 [==============================] - 1s 825us/step - loss: 0.2336 - acc: 0.7648
Epoch 42/80
1152/1152 [==============================] - 1s 817us/step - loss: 0.2245 - acc: 0.7778
Epoch 43/80
1152/1152 [==============================] - 1s 896us/step - loss: 0.2222 - acc: 0.7786
Epoch 44/80
1152/1152 [==============================] - 1s 828us/step - loss: 0.2237 - acc: 0.7769
Epoch 45/80
1152/1152 [==============================] - 1s 913us/step - loss: 0.2218 - acc: 0.7795
Epoch 46/80
1152/1152 [==============================] - 1s 917us/step - loss: 0.2164 - acc: 0.7847
Epoch 47/80
1152/1152 [==============================] - 1s 880us/step - loss: 0.2169 - acc: 0.7839
Epoch 48/80
1152/1152 [==============================] - 1s 872us/step - loss: 0.2160 - acc: 0.7847
Epoch 49/80
1152/1152 [==============================] - 1s 866us/step - loss: 0.2163 - acc: 0.7856
Epoch 50/80


1152/1152 [==============================] - 1s 826us/step - loss: 0.2307 - acc: 0.7700
Epoch 39/100
1152/1152 [==============================] - 1s 797us/step - loss: 0.2255 - acc: 0.7752
Epoch 40/100
1152/1152 [==============================] - 1s 849us/step - loss: 0.2287 - acc: 0.7726
Epoch 41/100
1152/1152 [==============================] - 1s 872us/step - loss: 0.2249 - acc: 0.7752
Epoch 42/100
1152/1152 [==============================] - 1s 826us/step - loss: 0.2294 - acc: 0.7708
Epoch 43/100
1152/1152 [==============================] - 1s 879us/step - loss: 0.2222 - acc: 0.7786
Epoch 44/100
1152/1152 [==============================] - 1s 837us/step - loss: 0.2259 - acc: 0.7760
Epoch 45/100
1152/1152 [==============================] - 1s 890us/step - loss: 0.2242 - acc: 0.7769
Epoch 46/100
1152/1152 [==============================] - 1s 863us/step - loss: 0.2324 - acc: 0.7682
Epoch 47/100
1152/1152 [==============================] - 1s 844us/step - loss: 0.2275 - acc: 0.7726
Epo

1152/1152 [==============================] - 1s 621us/step - loss: 0.2563 - acc: 0.7674
Epoch 16/20
1152/1152 [==============================] - 1s 560us/step - loss: 0.2462 - acc: 0.7778
Epoch 17/20
1152/1152 [==============================] - 1s 580us/step - loss: 0.2449 - acc: 0.7760
Epoch 18/20
1152/1152 [==============================] - 1s 573us/step - loss: 0.2440 - acc: 0.7752
Epoch 19/20
1152/1152 [==============================] - 1s 578us/step - loss: 0.2360 - acc: 0.7804
Epoch 20/20
1152/1152 [==============================] - 1s 583us/step - loss: 0.2343 - acc: 0.7804
{'bs:4, ep:20 (cm)': array([[5, 0],
       [3, 4]]), 'bs:4, ep:20 (accuracy)': '75.0%', 'bs:4, ep:40 (cm)': array([[4, 1],
       [3, 4]]), 'bs:4, ep:40 (accuracy)': '66.7%', 'bs:4, ep:60 (cm)': array([[4, 1],
       [3, 4]]), 'bs:4, ep:60 (accuracy)': '66.7%', 'bs:4, ep:80 (cm)': array([[4, 1],
       [3, 4]]), 'bs:4, ep:80 (accuracy)': '66.7%', 'bs:4, ep:100 (cm)': array([[3, 2],
       [3, 4]]), 'bs:4, ep:

...

1152/1152 [==============================] - 1s 464us/step - loss: 0.2113 - acc: 0.7891
Epoch 89/100
1152/1152 [==============================] - 1s 447us/step - loss: 0.2099 - acc: 0.7899
Epoch 90/100
1152/1152 [==============================] - 1s 453us/step - loss: 0.2082 - acc: 0.7917
Epoch 91/100
1152/1152 [==============================] - 1s 443us/step - loss: 0.2055 - acc: 0.7960
Epoch 92/100
1152/1152 [==============================] - 1s 449us/step - loss: 0.2077 - acc: 0.7934
Epoch 93/100
1152/1152 [==============================] - 0s 428us/step - loss: 0.2097 - acc: 0.7908
Epoch 94/100
1152/1152 [==============================] - 0s 430us/step - loss: 0.2123 - acc: 0.7882
Epoch 95/100
1152/1152 [==============================] - 1s 460us/step - loss: 0.2101 - acc: 0.7908
Epoch 96/100
1152/1152 [==============================] - 1s 475us/step - loss: 0.2059 - acc: 0.7951
Epoch 97/100
1152/1152 [==============================] - 1s 448us/step - loss: 0.2105 - acc: 0.7908
Epo

1152/1152 [==============================] - 0s 397us/step - loss: 0.2479 - acc: 0.7752
Epoch 24/40
1152/1152 [==============================] - 0s 385us/step - loss: 0.2465 - acc: 0.7734
Epoch 25/40
1152/1152 [==============================] - 0s 396us/step - loss: 0.2459 - acc: 0.7734
Epoch 26/40
1152/1152 [==============================] - 0s 356us/step - loss: 0.2395 - acc: 0.7778
Epoch 27/40
1152/1152 [==============================] - 0s 370us/step - loss: 0.2359 - acc: 0.7804
Epoch 28/40
1152/1152 [==============================] - 0s 372us/step - loss: 0.2358 - acc: 0.7795
Epoch 29/40
1152/1152 [==============================] - 0s 393us/step - loss: 0.2371 - acc: 0.7760
Epoch 30/40
1152/1152 [==============================] - 0s 395us/step - loss: 0.2323 - acc: 0.7812
Epoch 31/40
1152/1152 [==============================] - 0s 395us/step - loss: 0.2289 - acc: 0.7839
Epoch 32/40
1152/1152 [==============================] - 0s 393us/step - loss: 0.2319 - acc: 0.7786
Epoch 33/40


1152/1152 [==============================] - 0s 395us/step - loss: 0.2124 - acc: 0.7899
Epoch 51/60
1152/1152 [==============================] - 1s 443us/step - loss: 0.2079 - acc: 0.7951
Epoch 52/60
1152/1152 [==============================] - 0s 374us/step - loss: 0.2098 - acc: 0.7934
Epoch 53/60
1152/1152 [==============================] - 0s 408us/step - loss: 0.2079 - acc: 0.7951
Epoch 54/60
1152/1152 [==============================] - 0s 387us/step - loss: 0.2037 - acc: 0.8003
Epoch 55/60
1152/1152 [==============================] - 0s 393us/step - loss: 0.2064 - acc: 0.7969
Epoch 56/60
1152/1152 [==============================] - 0s 426us/step - loss: 0.2087 - acc: 0.7943
Epoch 57/60
1152/1152 [==============================] - 0s 384us/step - loss: 0.2122 - acc: 0.7917
Epoch 58/60
1152/1152 [==============================] - 0s 411us/step - loss: 0.2100 - acc: 0.7934
Epoch 59/60
1152/1152 [==============================] - 0s 385us/step - loss: 0.2097 - acc: 0.7917
Epoch 60/60


1152/1152 [==============================] - 1s 438us/step - loss: 0.2272 - acc: 0.7769
Epoch 57/80
1152/1152 [==============================] - 1s 441us/step - loss: 0.2271 - acc: 0.7752
Epoch 58/80
1152/1152 [==============================] - 0s 433us/step - loss: 0.2254 - acc: 0.7778
Epoch 59/80
1152/1152 [==============================] - 1s 435us/step - loss: 0.2230 - acc: 0.7795
Epoch 60/80
1152/1152 [==============================] - 0s 378us/step - loss: 0.2249 - acc: 0.7778
Epoch 61/80
1152/1152 [==============================] - 0s 374us/step - loss: 0.2280 - acc: 0.7734
Epoch 62/80
1152/1152 [==============================] - 0s 384us/step - loss: 0.2252 - acc: 0.7778
Epoch 63/80
1152/1152 [==============================] - 0s 395us/step - loss: 0.2273 - acc: 0.7743
Epoch 64/80
1152/1152 [==============================] - 0s 390us/step - loss: 0.2264 - acc: 0.7752
Epoch 65/80
1152/1152 [==============================] - 0s 388us/step - loss: 0.2268 - acc: 0.7743
Epoch 66/80


1152/1152 [==============================] - 0s 402us/step - loss: 0.2155 - acc: 0.7917
Epoch 42/100
1152/1152 [==============================] - 0s 372us/step - loss: 0.2126 - acc: 0.7934
Epoch 43/100
1152/1152 [==============================] - 0s 396us/step - loss: 0.2113 - acc: 0.7951
Epoch 44/100
1152/1152 [==============================] - 0s 379us/step - loss: 0.2120 - acc: 0.7925
Epoch 45/100
1152/1152 [==============================] - 0s 360us/step - loss: 0.2133 - acc: 0.7908
Epoch 46/100
1152/1152 [==============================] - 0s 398us/step - loss: 0.2113 - acc: 0.7934
Epoch 47/100
1152/1152 [==============================] - 0s 404us/step - loss: 0.2120 - acc: 0.7925
Epoch 48/100
1152/1152 [==============================] - 0s 377us/step - loss: 0.2122 - acc: 0.7934
Epoch 49/100
1152/1152 [==============================] - 0s 395us/step - loss: 0.2144 - acc: 0.7899
Epoch 50/100
1152/1152 [==============================] - 0s 411us/step - loss: 0.2174 - acc: 0.7856
Epo